In [222]:
import pandas as pd
import numpy as np
from phik.report import plot_correlation_matrix
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, \
    classification_report, roc_auc_score
from sklearn.preprocessing import PolynomialFeatures
import matplotlib.pyplot as plt
plt.style.use('dark_background')

In [334]:
df_raw = pd.read_csv('/kaggle/input/leopard-challenge-classification/train.csv')
test = pd.read_csv(r'/kaggle/input/leopard-challenge-classification/test.csv')
df_raw.shape, test.shape

((13863, 26), (5942, 25))

In [353]:
test_cv = test.drop(['oral', 'ID'], axis = 1).copy()
test_cv['tartar'] = test_cv['tartar'].apply(lambda x: 1 if x == 'Y' else 0)

In [354]:
merged_data = df_raw.drop(['oral', 'ID'], axis=1).copy()
merged_data['tartar'] = merged_data['tartar'].apply(lambda x: 1 if x == 'Y' else 0)

In [355]:
# Создание бинарного признака на основе BMI
merged_data['BMI'] = merged_data['weight(kg)'] / ((merged_data['height(cm)'] / 100) ** 2)
merged_data['BMI_status'] = (merged_data['BMI'] < 18.5) | (merged_data['BMI'] >= 24.9)
merged_data['BMI_status'] = merged_data['BMI_status'].astype(int)
merged_data['log_tr'] = merged_data['triglyceride'].apply(np.log)
merged_data['log_gtp'] = merged_data['Gtp'].apply(np.log)
merged_data['log_log_alt'] = np.log(np.log(merged_data['ALT']))
merged_data['BMI_log_gtp'] = merged_data['BMI'].apply(np.log)
merged_data = merged_data.drop(['triglyceride', 'Gtp', 'ALT', 'BMI'], axis=1)

test_cv['BMI'] = test_cv['weight(kg)'] / ((test_cv['height(cm)'] / 100) ** 2)
test_cv['BMI_status'] = (test_cv['BMI'] < 18.5) | (test_cv['BMI'] >= 24.9)
test_cv['BMI_status'] = test_cv['BMI_status'].astype(int)
test_cv['log_tr'] = test_cv['triglyceride'].apply(np.log)
test_cv['log_gtp'] = test_cv['Gtp'].apply(np.log)
test_cv['log_log_alt'] = np.log(np.log(test_cv['ALT']))
test_cv['BMI_log_gtp'] = test_cv['BMI'].apply(np.log)
test_cv = test_cv.drop(['triglyceride', 'Gtp', 'ALT', 'BMI'], axis=1)

In [356]:
from sklearn.model_selection import train_test_split
polynom = PolynomialFeatures(interaction_only=True, include_bias=False)

X = merged_data.drop(['smoking'], axis=1) 
Y = merged_data['smoking']

# X_poly = pd.DataFrame(polynom.fit_transform(X))
# test_cv_poly = pd.DataFrame(polynom.transform(test_cv))

X_new_train, X_test_sim, Y_new_train, Y_test_sim = train_test_split(X, Y, test_size=0.3, random_state=42, stratify=Y)
X_new_train.shape, X_test_sim.shape, Y_new_train.shape, Y_test_sim.shape

((9704, 25), (4159, 25), (9704,), (4159,))

In [357]:
# X_exper = X_poly.join(Y)

In [358]:
# table = X_exper\
#     .phik_matrix()['smoking'].to_frame()\
#     .sort_values('smoking', ascending=False).round(4)

In [359]:
# strong_cols = table.iloc[1:101, :].index.to_list()
# strong_cols

In [360]:
# X_poly = X_poly[strong_cols]
# test_cv_poly = test_cv_poly[strong_cols]
# X_poly.shape, test_cv_poly.shape

In [361]:
from sklearn.preprocessing import MinMaxScaler

# Масштабирование признаков
scaler = MinMaxScaler()

X_scale = scaler.fit_transform(X_new_train)
test_cv = scaler.transform(X_test_sim)

X = pd.DataFrame(X_scale)
X.shape, test_cv.shape

((9704, 25), (4159, 25))

In [362]:
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder, FunctionTransformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, GridSearchCV, RepeatedStratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve, RocCurveDisplay, cohen_kappa_score, log_loss, f1_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import RFE, RFECV
from sklearn.isotonic import IsotonicRegression
from sklearn.calibration import CalibrationDisplay
from sklearn.inspection import PartialDependenceDisplay, permutation_importance
from sklearn.linear_model import LogisticRegression
from collections import Counter
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

from sklearn.manifold import TSNE
import optuna

In [366]:
%%time
ens_cv_scores, ens_preds = list(), list()
hill_ens_cv_scores, hill_ens_preds =  list(), list()

ens_cv_scores = []
ens_preds = []

sk = RepeatedStratifiedKFold(n_splits = 7, n_repeats = 2, random_state = 42)
ratio = float(np.sum(Y == 0)) / np.sum(Y == 1)

for i, (train_idx, test_idx) in enumerate(sk.split(X, Y_new_train)):

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    Y_train, Y_test = Y_new_train.iloc[train_idx], Y_new_train.iloc[test_idx]
    X_train_sub, X_val_sub, Y_train_sub, Y_val_sub = train_test_split(X_train, Y_train, test_size=0.2, random_state=42, stratify=Y_train)
    
    print('----------------------------------------------------------')

    ##########
    ## LGBM ##
    ##########

    LGBM_md = LGBMClassifier(objective = 'binary',
                             class_weight='balanced',
                             n_estimators = 3500,
                             max_depth = 9,
                             learning_rate = 0.03,
                             num_leaves = 20,
                             reg_alpha = 2,
                             reg_lambda = 5,
                             subsample = 0.7,
                             colsample_bytree = 0.7).fit(X_train_sub, Y_train_sub, 
                                                            eval_set=[(X_val_sub, Y_val_sub)], 
                                                            early_stopping_rounds=100, 
                                                            verbose=100)

    lgb_pred = LGBM_md.predict_proba(X_test)[:, 1]
    lgb_score = roc_auc_score(Y_test, lgb_pred)

    print('Fold', i, '==> LGBM oof ROC-AUC score is ==>', lgb_score) 

    lgb_pred_test = LGBM_md.predict_proba(test_cv)[:, 1]

    #########
    ## XGB ##
    #########

    XGB_md = XGBClassifier(objective = 'binary:logistic',
                           tree_method = 'hist',
                           colsample_bytree = 0.7, 
                           gamma = 2, 
                           learning_rate = 0.01, 
                           max_depth = 10, 
                           min_child_weight = 10, 
                           n_estimators = 2500, 
                           reg_lambda=5,
                           scale_pos_weight=ratio,
                           subsample = 0.7).fit(X_train_sub, Y_train_sub, 
                                                   eval_metric="auc", 
                                                   eval_set=[(X_val_sub, Y_val_sub)], 
                                                   early_stopping_rounds=150, 
                                                   verbose=100)

    xgb_pred = XGB_md.predict_proba(X_test)[:, 1]
    xgb_score = roc_auc_score(Y_test, xgb_pred)

    print('Fold', i, '==> XGB oof ROC-AUC score is ==>', xgb_score)

    xgb_pred_test = XGB_md.predict_proba(test_cv)[:, 1]

    ##############
    ## CatBoost ##
    ##############

    Cat_md = CatBoostClassifier(auto_class_weights='Balanced',
                                eval_metric='AUC',
                                iterations = 2500,
                                learning_rate = 0.01,
                                depth = 8,
                                random_strength = 0.5,
                                bagging_temperature = 0.7,
                                border_count = 30,
                                l2_leaf_reg = 5,
                                verbose = False, 
                                task_type = 'CPU').fit(X_train_sub, Y_train_sub, 
                                                           eval_set=(X_val_sub, Y_val_sub), 
                                                           early_stopping_rounds=150, 
                                                           verbose=100, 
                                                           use_best_model=True)

    cat_pred = Cat_md.predict_proba(X_test)[:, 1]
    cat_score = roc_auc_score(Y_test, cat_pred)

    print('Fold', i, '==> CatBoost oof ROC-AUC score is ==>', cat_score)

    cat_pred_test = Cat_md.predict_proba(test_cv)[:, 1]    
    
    ##############
    ## Ensemble ##
    ##############
    
    
    print('Fold', i, '==> Average Ensemble oof ROC-AUC score is ==>', ens_score_fold)
    
    # Ансамбль моделей
    ens_pred_1 = ( lgb_pred + xgb_pred + cat_pred ) / 3
    ens_pred_2 = ( lgb_pred_test + xgb_pred_test + cat_pred_test ) / 3
    ens_score_fold = roc_auc_score(Y_test, ens_pred_1)
    ens_cv_scores.append(ens_score_fold)
    ens_preds.append(ens_pred_2)

    # Поиск оптимального порога для F1
    thresholds = np.linspace(0, 1, 300)
    f1_scores = [f1_score(Y_test, ens_pred_1 > thresh) for thresh in thresholds]
    optimal_threshold = thresholds[np.argmax(f1_scores)]
    ens_f1_score = max(f1_scores)

ens_mean_score = np.mean(ens_cv_scores)
optimal_threshold, ens_mean_score

----------------------------------------------------------


/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	valid_0's binary_logloss: 0.613776
[200]	valid_0's binary_logloss: 0.588645
[300]	valid_0's binary_logloss: 0.572096
[400]	valid_0's binary_logloss: 0.561181
[500]	valid_0's binary_logloss: 0.552708
[600]	valid_0's binary_logloss: 0.545111
[700]	valid_0's binary_logloss: 0.538404
[800]	valid_0's binary_logloss: 0.531715
[900]	valid_0's binary_logloss: 0.52683
[1000]	valid_0's binary_logloss: 0.521617
[1100]	valid_0's binary_logloss: 0.51672
[1200]	valid_0's binary_logloss: 0.5128
[1300]	valid_0's binary_logloss: 0.508855
[1400]	valid_0's binary_logloss: 0.504863
[1500]	valid_0's binary_logloss: 0.50139
[1600]	valid_0's binary_logloss: 0.499353
[1700]	valid_0's binary_logloss: 0.496953
[1800]	valid_0's binary_logloss: 0.494454
[1900]	valid_0's binary_logloss: 0.493153
[2000]	valid_0's binary_logloss: 0.492527
[2100]	valid_0's binary_logloss: 0.491723
[2200]	valid_0's binary_logloss: 0.491123
[2300]	valid_0's binary_logloss: 0.490816
[2400]	valid_0's binary_logloss: 0.489855
[2500]

/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[100]	validation_0-auc:0.71606
[200]	validation_0-auc:0.72438
[300]	validation_0-auc:0.72645
[400]	validation_0-auc:0.72773
[500]	validation_0-auc:0.73015
[600]	validation_0-auc:0.73094
[700]	validation_0-auc:0.73202
[800]	validation_0-auc:0.73076
[855]	validation_0-auc:0.73132
Fold 0 ==> XGB oof ROC-AUC score is ==> 0.711843463672732
0:	test: 0.6300929	best: 0.6300929 (0)	total: 2.73ms	remaining: 6.83s
100:	test: 0.6940799	best: 0.6940799 (100)	total: 292ms	remaining: 6.93s
200:	test: 0.7094875	best: 0.7094875 (200)	total: 579ms	remaining: 6.63s
300:	test: 0.7162579	best: 0.7165425 (298)	total: 883ms	remaining: 6.45s
400:	test: 0.7202695	best: 0.7205182 (399)	total: 1.19s	remaining: 6.22s
500:	test: 0.7235101	best: 0.7235773 (497)	total: 1.49s	remaining: 5.96s
600:	test: 0.7254935	best: 0.7254935 (600)	total: 1.8s	remaining: 5.7s
700:	test: 0.7256347	best: 0.7260470 (689)	total: 2.11s	remaining: 5.42s
800:	test: 0.7262779	best: 0.7265154 (796)	total: 2.41s	remaining: 5.12s
900:	test: 

...

/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	valid_0's binary_logloss: 0.616945
[200]	valid_0's binary_logloss: 0.598139
[300]	valid_0's binary_logloss: 0.58828
[400]	valid_0's binary_logloss: 0.580938
[500]	valid_0's binary_logloss: 0.573791
[600]	valid_0's binary_logloss: 0.567426
[700]	valid_0's binary_logloss: 0.561764
[800]	valid_0's binary_logloss: 0.556275
[900]	valid_0's binary_logloss: 0.55165
[1000]	valid_0's binary_logloss: 0.547858
[1100]	valid_0's binary_logloss: 0.54432
[1200]	valid_0's binary_logloss: 0.541828
[1300]	valid_0's binary_logloss: 0.539463
[1400]	valid_0's binary_logloss: 0.537359
[1500]	valid_0's binary_logloss: 0.536198
[1600]	valid_0's binary_logloss: 0.534727
[1700]	valid_0's binary_logloss: 0.533854
[1800]	valid_0's binary_logloss: 0.532378
[1900]	valid_0's binary_logloss: 0.532552
Fold 13 ==> LGBM oof ROC-AUC score is ==> 0.6768696719194007
[0]	validation_0-auc:0.61622


/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[100]	validation_0-auc:0.67041
[200]	validation_0-auc:0.67590
[300]	validation_0-auc:0.67758
[400]	validation_0-auc:0.68022
[500]	validation_0-auc:0.68043
[600]	validation_0-auc:0.67997
[638]	validation_0-auc:0.68051
Fold 13 ==> XGB oof ROC-AUC score is ==> 0.708996383363472
0:	test: 0.6000486	best: 0.6000486 (0)	total: 3.09ms	remaining: 7.71s
100:	test: 0.6546162	best: 0.6553894 (95)	total: 279ms	remaining: 6.63s
200:	test: 0.6697370	best: 0.6697370 (200)	total: 568ms	remaining: 6.49s
300:	test: 0.6783742	best: 0.6784011 (298)	total: 868ms	remaining: 6.34s
400:	test: 0.6805929	best: 0.6811554 (390)	total: 1.15s	remaining: 6.04s
500:	test: 0.6819622	best: 0.6819622 (500)	total: 1.46s	remaining: 5.83s
600:	test: 0.6808349	best: 0.6819622 (500)	total: 1.77s	remaining: 5.6s
Stopped by overfitting detector  (150 iterations wait)

bestTest = 0.6819622239
bestIteration = 500

Shrink model to first 501 iterations.
Fold 13 ==> CatBoost oof ROC-AUC score is ==> 0.7033550762076982
Fold 13 ==> Av

(0.42474916387959866, 0.7093892116111749)

In [367]:
avg_ens_preds = np.mean(ens_preds, axis=0)
final_predictions = (avg_ens_preds > optimal_threshold).astype(int)
final_predictions.sum()

1265

In [368]:
final_predictions

array([1, 0, 0, ..., 1, 1, 1])

In [369]:
from sklearn.metrics import confusion_matrix

# Теперь вы можете сравнить прогнозы с реальными значениями
cm = confusion_matrix(Y_test_sim, final_predictions)

cm

array([[2530,  789],
       [ 364,  476]])

In [241]:
submission = pd.read_csv(r'/kaggle/input/leopard-challenge-classification/sample_submission.csv')

In [243]:
submission['smoking'] = final_predictions
submission.to_csv('Ensemble_submission_05.csv', index = False)

In [245]:
submission['smoking'].sum()

1862